# Sistemas de Recomendación basados en SVD

## SVD: Siempre venciendo desafios

### Matemática Numérica
### Facultad de Matemática y Computación

Autores:
- Daniel Abad Fundora C212
- Anabel Benítez González C211
- Raudel Alejandro Gómez Molina C211
- Alex Sierra Alcalá C211

# Sistemas de Recomendación

Desde el nacimiento de internet como tecnología se han abierto nuevas posibilidades que facilitan el acceso a la información, también esta se ha aumentado considerablemente. Tal impacto se ha visto reflejado en todo tipo de áreas, una de ellas es el comercio, donde sitios especializados en internet ofertan millones de productos o servicios, también hay sitios donde la 
mayoría de datos pueden ser efímeros o de poca utilidad, produciendo un caos de información que, al contrario de satisfacer requerimientos de los usuarios, pueden complicar más su respuesta; para que sea efectiva, depende de la habilidad y experiencia que tenga el usuario para expresar sus necesidades mediante una consulta y está demostrado que en la mayoría de casos es imprecisa y vaga. Por lo anterior surgen los sistemas de recuperación de información, que si bien apuntan a mejorar la forma en que se almacena, representa y organiza la información, su función no es devolver la información deseada por el usuario sino únicamente indicar qué documentos son potencialmente relevantes para dicha necesidad de información. A partir de esto, surge una nueva necesidad de encontrar herramientas que se encarguen de buscar por los usuarios, que los conozcan y recomienden un conjunto limitado de opciones acordes a sus intereses. 

Un sistema de recomendación se puede definir, de manera formal, como aquel sistema que tiene como principal tarea seleccionar ciertos objetos de acuerdo con los requerimientos del usuario. Estos sistemas son muy atractivos en situaciones donde la cantidad de información que se ofrece al usuario supera ampliamente cualquier capacidad individual de exploración.

La creación de un sistema de recomendación cuenta con tres fases principales:
- Captura de las preferencias y los gustos e intereses del usuario.
- Extracción del conocimiento, aquí el sistema se encarga de interpretar la informacion que recopiló anteriormente para poder predecir los gustos y las preferencias del usuario.
- A partir del contenido procesado anteriormente el sistema se encarga de se seleccionar los ítems que podrían interesarle a un determinado usario.

Ahora debemos encontrar un mecanismo para procesar toda la información que recopilemos previamente de los usuarios, para ello debemos reducir la dimensionalidad de nuestra matriz de votaciones, ya que esta puede ser muy grande y dispersa, a este proceso lo llamareos \textit{reducción de la dimensinalidad}. 

Por lo que es necesario determinar una matriz que sea equivalente a la original y que sea mas concisa a la hora de brindar la información para realizar la recomendación. Esto permite hacer más eficiente el proceso pues solo tendremos que considerar las características principales en vez de analizar completamente toda nuestra extensa matriz original, además esto permite minimizar los problemas relacionados con la presencia de datos erróneos en nuestra recopilación.

# SVD (Singular Value Descomposition)

La descomposición singular de valores es una 
técnica de factorización de matrices que permite 
descomponer una matriz de dimensión nxm A en otras matrices U,
S, V tales que :

$$ 
A = U \cdot \Sigma \cdot  V^T  
$$

Donde :
* *U* de tamaño (n, n) es una matriz ortogonal que contiene los vectores singulares izquierdos de *A*.
* En $\Sigma$ que es una matriz diagonal (n,d), cuyos valores son los valores singulares de la matriz *A* ordenados en valor decreciente
* En *V* que es una matriz transpuesta (d,d), cuyos valores son los vectores singulares derechos de *A*.

Existe una propiedad 
aplicada a SVD enfocados en los sistemas 
de recomendación, esta consiste en que 
reduciendo el número de valores singulares 
de la matriz $\Sigma$ a los primeros k valores, se 
obtendrá una aproximación de la matriz 
original A, que permite ser reconstruida a 
partir de las versiones reducidas de las otras 
matrices cometiendo un cierto error, pero 
disminuyendo el tamaño. Es decir: 

$$ 
A_{n, m} \approx U_{n, k} \cdot \Sigma_{k, k} \cdot V^T_{k, m}  
$$

Esta propiedad es derivada del 
teorema de Eckart-Young que aborda 
la mejor aproximación a la matriz original 
A, obteniéndola poniendo a 0 los n valores 
singulares más pequeños, así se reducirán 
las matrices al número de valores singulares 
no nulos que tenga la matriz $\Sigma$. Esto resulta 
entonces en la transformación de gran cantidad 
de datos en su representación reducida, siendo 
por lo tanto una propiedad muy importante 
que permite reducir considerablemente el 
tiempo de cómputo de cálculo y de uso de 
memoria para las tres matrices.

# Filtrado Colaborativo

El filtrado colaborativo es un método para hacer predicciones automáticas (filtrado) sobre los intereses de un usuario mediante la recopilación de las preferencias o gustos de información de muchos usuarios (colaborador). El Filtrado colaborativo se basa, en que si una persona A tiene la misma opinión que una persona B sobre un tema, A es más probable que tenga la misma opinión que B en otro tema diferente que la opinión que tendría una persona elegida azar. 

Dentro del filtrado colaborativo, es necesario el 
manejo de la matriz de votaciones de usuarios 
e ítems, por lo tanto, es posible considerar esta 
matriz como la base para aplicar SVD y obtener la 
factorización de matrices U, S y V. Luego, usando 
el teorema de Eckar-Youn, se puede reducir a k
dimensiones. La matriz de factorización sería:

$$ 
A_{usuarios, ítems} \approx U_{usuarios, k} \cdot \Sigma_{k, k} \cdot V^T_{k, ítems}
$$

Por otro lado, es posible simplificar 
el proceso de SVD obteniendo solo los 
factores de usuarios e ítems, esto es posible 
descomponiendo la matriz $\Sigma$ en dos matrices 
iguales, factores de usuarios *Ufac* y factores de 
ítems *Ifac* de esta manera:

$$
\Sigma_{k,k} = 
\left(\begin{array}{}
\lambda_{1} &  & ...\\
& \lambda_{2}\\
... \\
& & \lambda_{k}
\end{array}\right)
$$

$$
\Sigma_{k,k} = 
\left(\begin{array}{}
\sqrt{\lambda_{1}} &  & ...\\
& \sqrt{\lambda_{2}}\\
... \\
& & \sqrt{\lambda_{k}}
\end{array}\right) 
\cdot 
\left(\begin{array}{}
\sqrt{\lambda_{1}} &  & ...\\
& \sqrt{\lambda_{2}}\\
... \\
& & \sqrt{\lambda_{k}}
\end{array}\right)
$$

Nota : Los $\lambda_{i}$ son valores singulares de A.

Por tanto la matriz de votaciones se puede expresar como *Votos* = *Ufact* $\cdot$ *Ifact*, donde :

$$
Ufact = U_{n, k} \cdot
\left(\begin{array}{}
\sqrt{\lambda_{1}} &  & ...\\
& \sqrt{\lambda_{2}}\\
... \\
& & \sqrt{\lambda_{k}}
\end{array}\right)
$$


$$
Ifact =
\left(\begin{array}{}
\sqrt{\lambda_{1}} &  & ...\\
& \sqrt{\lambda_{2}}\\
... \\
& & \sqrt{\lambda_{k}}
\end{array}\right)
\cdot
V^T_{k, m}  
$$

Para hallar esta aproximación, la idea es buscar los 
factores de usuarios e ítems a partir de la matriz 
de votación abordándolo como un problema de 
regresión, donde se quieren encontrar los valores 
de las matrices de factores. Así, si se 
considera a $q_{i}$ al vector que represente los 
factores de un ítem y $p_{j}$ al vector que representa 
los factores del usuario se tendría que cumplir :

$$
A_{i, j} = q^T_{i} \cdot p_{j} \approx \sum_{n = 1}^{k} q_{i, n}\cdot p_{n, j}
$$

Ahora solo se ajustan los factores de 
aquellos usuarios que hayan definido un voto 
reduciendo considerablemente el sistema de 
ecuaciones y solucionando el problema de la 
dispersión, cometiendo un pequeño error $e_{i, j}$ = |$A_{i, j}$ - $q^T_{i} \cdot p_{j}$|. Debido a que el error absoluto es una 
función complicada de tratar, se usará el error 
cuadrado medio:

$$
(e_{i, j})^2 = (A_{i, j} - q^T_{i} \cdot p_{j})^2 \approx (A_{i, j} - \sum q_{i, n}\cdot p_{n, j})^2
$$

O sea, intentar encontrar el error mínimo para hacer la mejor aproximación de la siguiente forma:

$$
min_{p, q} = \sum (A_{i, j} - q_{i, n}\cdot p_{n, j})^2
$$

Pero es necesario además hacer una regularización para evitar el *Outfitting*, así, es posible llegar a:

$$
(e_{i, j})^2 = (A_{i, j} - \sum q_{i, n}\cdot p_{n, j})^2 + \frac{\lambda}{2} \cdot (||q_{i}||^2 + ||p_{j}||^2) \space (1)
$$

Donde $\lambda$ es una constante de regularización.

Para encontrar el mínimo de la función ya detallada 
existen técnicas como la del descenso del gradiente que 
buscan ajustar las matrices de factores poco a 
poco de manera automática. Esto se realizará en la 
función (1), con el fin de encontrar el mínimo de 
la función, o acercarse lo suficiente a la solución. Esta técnica consiste en ir 
moviéndose poco a poco por la función hasta encontrar el mínimo evaluando, la función en un punto dado para 
luego moverse una distancia hacia otro punto; 
ahora se calcula la derivada en ese punto y 
se desplaza al lado opuesto de la derivada de 
la función que se quiere minimizar.

Aplicando descenso del gradiente en (1) se tiene:

$$
\frac{\delta(e_{i, j})^2}{\delta p_{i, n}} = -2q_{n, j}\cdot (A_{i, j} - \sum q_{i, n}\cdot p_{n, j}) + \lambda p_{i, n} \space (2)
$$

$$
\frac{\delta(e_{i, j})^2}{\delta q_{n, j}} = -2p_{i, n}\cdot (A_{i, j} - \sum q_{i, n}\cdot p_{n, j}) + \lambda q_{n, j} \space (3)
$$

Teniendo ya el gradiente, solo queda 
aplicar las reglas de actualización tanto para $p_{i, n}$
como para $q_{n, j}$ en (2), (3) con las constantes de 
regularización y aprendizaje. Así, la regla de 
actualización parte de $\sigma_{n + 1} = \sigma_{n} - \alpha \nabla f(x)$, donde $\sigma_{n + 1}$ es el nuevo valor en la matriz de votaciones, $\sigma_{n}$ su valor actual, $\alpha$ constante de aprendizaje y $\nabla f(x)$ es el gradiente ya obtenido. Dando como resultado :

Para $p_{i, n}$ :

$$
p_{i, n}^{'} = p_{i, n} + \alpha[2q_{n, j}\cdot (A_{i, j} - \sum q_{i, n}\cdot p_{n, j}) - \lambda p_{i, n}] \space (4)
$$

Para $p_{n, j}$ :

$$
q_{n, j}^{'} = q_{n, j} + \alpha[2p_{i, n}\cdot (A_{i, j} - \sum q_{i, n}\cdot p_{n, j}) - \lambda q_{n, j}] \space (5)
$$

El enfoque se plasma en las expresiones (4) y 
(5) que serán las que actualizarán los valores de 
la matriz de votaciones a partir de cada *Ephochs*, 
tanto para los factores de usuarios, como para los 
factores de ítems.


## Aplicación práctica

Lo que se hace con los motores de recomendación, es para una película que tu no has visto, teniendo en cuenta tus características y las de otros usuarios. Mediante SVD nos quedamos con los usuarios que son parecidos a ti, y vemos las peliculas que no has visto
## Preprocesamos los datos

###  Cargar Bibliotecas

In [1]:
import pandas as pd
from pandas.core.frame import DataFrame
from pandas.io.parsers import read_csv
from surprise import SVDpp
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
from collections import defaultdict

### Cargar Datos 

In [2]:
items = pd.read_csv("data/items.csv")
ratings = pd.read_csv("data/ratings.csv")
df_items = ratings.merge(items, on="itemId", how="left")
df_items_to_model = df_items[df_items.columns[:3]]

### Preprocesamiento

In [3]:
reader = Reader()
data = Dataset.load_from_df(df_items_to_model[df_items.columns[:3]], reader)
train, test = train_test_split(data, test_size=0.25)


### Entrenamiento y testing

In [4]:
svd = SVDpp()
svd.fit(train)
preds = svd.test(test)

### Evaluación

It's a good evaluations marks so let's train the model with the complete Dataset

In [5]:
accuracy.mae(preds)
accuracy.rmse(preds)

MAE:  0.6637
RMSE: 0.8647


0.8647039388289505

### Entrenar todos los datos

In [6]:
trainfull = data.build_full_trainset()

svd = SVDpp()
svd.fit(trainfull)

svd.predict(uid=1, iid=1)

Prediction(uid=1, iid=1, r_ui=None, est=4.6197697754454365, details={'was_impossible': False})

### Función de Recomendación

In [7]:
def recommend_system(userId, dataframe, algorithm, n_commends):
    """
with the parameters, returns back the top n recommends items.

Parameters
-----------

userId: the user ID of the person that we want recommendations

dataframe: the DataFrame of items.

algorithm: the algorith used to recommend items.

n_commends: the number of items recommended.


return
------

ID of items that a specific user will like.

    """
    item_ids = dataframe['itemId'].to_list()
    items_watched = dataframe[dataframe["userId"] == userId]["itemId"]
    items_no_watched = [item for item in item_ids if item not in items_watched]

    preds = [algorithm.predict(uid=userId, iid=movie) for movie in items_no_watched]
    commends_ratting = {pred[1]:pred[3] for pred in preds}
    order_dict = {k: v for k, v in sorted(commends_ratting.items(), key=lambda item: item[1])}

    top_predictions = list(order_dict.keys())[:n_commends]

    return dataframe[dataframe["itemId"].isin(top_predictions)][["title", "genres"]].drop_duplicates()

In [8]:
items_recommended = recommend_system(3, df_items, svd, 5)
print("ID of the items recommended:", items_recommended)

ID of the items recommended:                                  title                          genres
1197                   Twilight (2008)  Drama|Fantasy|Romance|Thriller
2791   Blair Witch Project, The (1999)           Drama|Horror|Thriller
10294  Witches of Eastwick, The (1987)  Comedy|Fantasy|Horror|Thriller
14369               Spice World (1997)                          Comedy
23088                Jack Frost (1998)           Children|Comedy|Drama


### Comprobar los resultados

In [9]:
def check_items_user(userId, dataframe, n):
    return dataframe[dataframe["userId"] ==userId].sort_values("rating", ascending=False)[:n]

In [10]:
print(f"Items user likes:", check_items_user(1, df_items, 20))

Items user likes:      userId  itemId  rating  timestamp  \
231       1    5060     5.0  964984002   
185       1    2872     5.0  964981680   
89        1    1291     5.0  964981909   
90        1    1298     5.0  964984086   
190       1    2948     5.0  964982191   
189       1    2947     5.0  964982176   
188       1    2944     5.0  964981872   
186       1    2899     5.0  964982703   
184       1    2858     5.0  964980868   
179       1    2700     5.0  964980985   
98        1    1517     5.0  964981107   
100       1    1573     5.0  964982290   
102       1    1587     5.0  964982346   
103       1    1617     5.0  964982951   
181       1    2761     5.0  964982703   
105       1    1625     5.0  964983504   
88        1    1282     5.0  964982703   
87        1    1278     5.0  964983414   
86        1    1275     5.0  964982290   
85        1    1270     5.0  964983705   

                                                 title  \
231                       M*A*S*H (a.k.a.